In [1]:
using Printf
using ArgParse
using Logging

using Dates
using CSV
using NPZ
using JSON
using HDF5
using DataStructures
using DelimitedFiles
using DataFrames
using NetCDF

In [2]:
base_folder = ".."

include(joinpath(base_folder, "MMCAcovid19_vac/markov_vac_aux.jl"))
include(joinpath(base_folder, "MMCAcovid19_vac/markov_vac_io.jl"))
include(joinpath(base_folder, "MMCAcovid19_vac/markov_vac.jl"))

run_epidemic_spreading_mmca!

In [79]:
data_path       = "../data/"
config_fname    = "../test/config_test.json"
config          = JSON.parsefile(config_fname);
data_dict       = config["data"]
pop_params_dict = config["population_params"]

# Reading metapopulation Dataframe
metapop_data_filename = joinpath(data_path, data_dict["metapopulation_data_filename"])
metapop_df            = CSV.read(metapop_data_filename, DataFrame)

# Metapopulations patches coordinates (labels)
M_coords = map(String, metapop_df[:, "id"])
M = length(M_coords)

# Coordinates for each age strata (labels)
G_coords = map(String, pop_params_dict["age_labels"])
G = length(G_coords)

# Coordinates for eacn vaccination status
V = 2
V_coords = ["NV", "V"]

N = 10
comp_coords = ["S", "E", "A", "I", "PH", "PD", "HR", "HD", "R", "D"]

# Loading initial 
A0_instance_filename = joinpath(data_path, "A0_initial_conditions.csv")

conditions₀ = CSV.read(A0_instance_filename, DataFrame);

seeds_age_ratio = [0.12, 0.16, 0.72] # Where this is comming from?


Y_ratio = sum(metapop_df.Y) / sum(metapop_df.Total)
M_ratio = sum(metapop_df.M) / sum(metapop_df.Total)
O_ratio = sum(metapop_df.O) / sum(metapop_df.Total)

[Y_ratio M_ratio Y_ratio]

1×3 Matrix{Float64}:
 0.191595  0.540813  0.191595

In [82]:
A₀_idx = 3
V_idx  = 1
M_idx  = Int.(conditions₀[:,"idx"])

compartments = zeros(Float64, G, M, V, N);
compartments[:, M_idx, V_idx, A₀_idx] .= seeds_age_ratio .* transpose(conditions₀[:,"seed"])

output_fname = "../test/initial_condition.nc"

isfile(output_fname) && rm(output_fname)
nccreate(output_fname, "data", "G", G_coords, "M", M_coords, "V", V_coords, "epi_states", comp_coords)
ncwrite(compartments, output_fname, "data")


# sum(transpose(seeds_age_ratio))

145.2